In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
!export AWS_SHARED_CREDENTIALS_FILE=/content/drive/My\ Drive/config/awscli.ini
path = "/content/drive/MyDrive/config/awscli.ini"
os.environ['AWS_SHARED_CREDENTIALS_FILE'] = path
print(os.environ['AWS_SHARED_CREDENTIALS_FILE'])

/content/drive/MyDrive/config/awscli.ini


In [1]:
%%time
import pyarrow.dataset as ds
dataset = ds.dataset("s3://ursa-labs-taxi-data/", partitioning=["year", "month"])
dataset

CPU times: user 906 ms, sys: 94.3 ms, total: 1e+03 ms
Wall time: 2.39 s


In [20]:
# * Which day (of week), on average has the highest tip?
# %%time
import pyarrow.compute as pc
import pyarrow.parquet as pq
from datetime import datetime
weekdays = ("Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday")
weekday_count = [0] * 7
weekdays_tips = [0] * 7
weekday_average = [0] * 7
columns_to_select =['dropoff_at', 'tip_amount']
for frag in dataset.get_fragments():
  # print("New Fragment")
  frag_table = frag.to_table(columns = columns_to_select)
  sorted_indices = pc.sort_indices([dt.as_py().weekday() for dt in frag_table["dropoff_at"]])
  sorted_table = frag_table.take(sorted_indices)

  row_group_max_size = frag_table.to_batches()[0].num_rows
  pq.write_table(sorted_table, 'optimized_parquet_file.parquet', row_group_size=row_group_max_size)
  optimized_parquet_file = pq.ParquetFile('optimized_parquet_file.parquet')

  for i in range(optimized_parquet_file.num_row_groups):
    # print("New Row Group")
    # name_2_pos = {x["path_in_schema"]:i for i, x in enumerate(optimized_parquet_file.metadata.row_group(i).to_dict()["columns"])}
    col_stats = optimized_parquet_file.metadata.row_group(i).column(0).statistics
    tip_amounts= optimized_parquet_file.read_row_group(i)["tip_amount"]
    # print("min : ", col_stats.min.weekday(),"max : ",col_stats.max.weekday())
    if col_stats.min.weekday() == col_stats.max.weekday():
        # print("IF")
        weekday_count[col_stats.min.weekday()] += len(tip_amounts)
        weekdays_tips[col_stats.min.weekday()] += pc.sum(tip_amounts).as_py()
        # print(" weekday_count ",len(tip_amounts))
        # print(" weekdays_tips ",pc.sum(tip_amounts).as_py())
    else:
      # print("ELSE")
      dropoffs= optimized_parquet_file.read_row_group(i)["dropoff_at"]
      for day in range(col_stats.min.weekday(),col_stats.max.weekday()+1):
        filtered_weekday = [dropoff_at.as_py().weekday() == day for dropoff_at in dropoffs]
        filtered_tip_amount = pc.filter(tip_amounts, filtered_weekday)
        if len(filtered_tip_amount)>0 :
          weekday_count[day] += len(filtered_tip_amount)
          weekdays_tips[day] += pc.sum(filtered_tip_amount).as_py()
          # print(" weekday_count ",len(filtered_tip_amount))
          # print(" weekdays_tips ",pc.sum(filtered_tip_amount).as_py())
    # print(weekday_count)
    # print(weekdays_tips)
  # break
for day in range(len(weekdays)):
  print("DAY ",day)
  if weekday_count[day]>0:
    weekday_average[day] = weekdays_tips[day]/weekday_count[day]
    print("Average tip of ",weekdays[day]," is ", weekday_average[day])
  else:
    print("We had no trip on ", weekday_count[day])


max_average_tip, max_day = max((value, index) for index, value in enumerate(weekday_average))
print (weekdays[max_day],", on average has the highest tip.")

DAY  0
Average tip of  Monday  is  1.3426116066510039
DAY  1
Average tip of  Tuesday  is  1.3530914447850408
DAY  2
Average tip of  Wednesday  is  1.3859836416488873
DAY  3
Average tip of  Thursday  is  1.396707128919658
DAY  4
Average tip of  Friday  is  1.3326901111708764
DAY  5
Average tip of  Saturday  is  1.1603224825390395
DAY  6
Average tip of  Sunday  is  1.285698632940804
Thursday , on average has the highest tip.
